In [38]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import pandas_ta as ta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import seaborn as sns

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

# feature engineering and scaling
def prep(dataset):
    scaler = StandardScaler()

    macro_path1 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\IRP DATASET  - Copy of US_macroeconomics.csv"
    macro_path2 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\^VIX.csv"
    macro_path3 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\CORESTICKM159SFRBATL.csv"
    macro_path4 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\FFR.csv"
    macro_path5 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GDP.csv"
    macro_path6 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VIX9D_History.csv"
    macro_path7 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GVZ_History.csv"
    macro_path8 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\OVX_History.csv"
    macro_path9 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VVIX_History.csv"
    macro_data1 = pd.read_csv(macro_path1)
    macro_data2 = pd.read_csv(macro_path2)
    macro_data3 = pd.read_csv(macro_path3)
    macro_data4 = pd.read_csv(macro_path4)
    macro_data5 = pd.read_csv(macro_path5)
    macro_data6 = pd.read_csv(macro_path6)
    macro_data7 = pd.read_csv(macro_path7)
    macro_data8 = pd.read_csv(macro_path8)
    macro_data9 = pd.read_csv(macro_path9)

    dataset['Date'] = pd.to_datetime(dataset['Date'], dayfirst=True)
    dataset.set_index('Date', inplace=True)
    dataset['Target'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

    macro_data1['date'] = pd.to_datetime(macro_data1['date'], dayfirst=True)
    macro_data1.set_index('date', inplace=True)

    macro_data2['Date'] = pd.to_datetime(macro_data2['Date'], dayfirst=True)
    macro_data2.set_index('Date', inplace=True)

    macro_data3['Date'] = pd.to_datetime(macro_data3['Date'], dayfirst=True)
    macro_data3.set_index('Date', inplace=True)

    macro_data4['Date'] = pd.to_datetime(macro_data4['Date'], dayfirst=True)
    macro_data4.set_index('Date', inplace=True)

    macro_data5['Date'] = pd.to_datetime(macro_data5['Date'], dayfirst=True)
    macro_data5.set_index('Date', inplace=True)

    macro_data7['Date'] = pd.to_datetime(macro_data7['Date'], dayfirst=False)
    macro_data7.set_index('Date', inplace=True)

    macro_data8['Date'] = pd.to_datetime(macro_data8['Date'], dayfirst=False)
    macro_data8.set_index('Date', inplace=True)

    macro_data9['Date'] = pd.to_datetime(macro_data9['Date'])
    macro_data9.set_index('Date', inplace=True)

    merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
    merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
    merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
    merge_data4 = merge_data3.join(macro_data4, how='left').fillna(method='ffill')
    merge_data5 = merge_data4.join(macro_data5, how='left').fillna(method='ffill')
    merge_data7 = merge_data5.join(macro_data7, how='left').fillna(method='ffill')
    merge_data8 = merge_data7.join(macro_data8, how='left').fillna(method='ffill')
    merge_final = merge_data8.join(macro_data9, how='left').fillna(method='ffill')

    merge_final['RSI (14D)'] = ta.rsi(merge_final['Close'], length=14)
    merge_final['20 Day CCI'] = ta.cci(high=merge_final['High'], low=merge_final['Low'], 
                                    close=merge_final['Close'], length=20)
    merge_final['Williams %R'] = ta.willr(high=merge_final['High'], low=merge_final['Low'], 
                                        close=merge_final['Close'], length=14)
    merge_final['EMA (5D)'] = merge_final['Close'].ewm(span=5, adjust=False).mean()
    merge_final['MA50'] = merge_final['Close'].rolling(window=50).mean()

    final_dataset = merge_final.dropna()

    time_shifted = final_dataset.resample('M').asfreq().dropna()

    features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50',]
    
    time_shifted[features] = time_shifted[features].astype(float)
    time_shifted[features] = scaler.fit_transform(time_shifted[features])

    return time_shifted, scaler

# get dataset
file_path = filedialog.askopenfilename(parent=root, title="Select A File")
ticker = pd.read_csv(file_path)
ticker, scaler = prep(ticker)

ticker

,Open,High,Low,Close,Adj Close,Volume,Target,CPI,Mortgage_rate,Unemp_rate,...,DFF,GDP,GVZ,OVX,VVIX,RSI (14D),20 Day CCI,Williams %R,EMA (5D),MA50
Date,,,,,,,,,,,,,,,,,,,,,
2009-09-30,-1.534147,-1.544658,-1.556026,-1.547917,-1.547917,2.052240,0.0,-1.707271,1.894013,1.460242,...,0.07,14448.882,0.802744,0.461573,-1.512091,0.402979,0.139903,-0.592440,-1.536879,-1.640859
2009-11-30,-1.475801,-1.482245,-1.478456,-1.473350,-1.473350,-0.247923,1.0,-1.662329,1.710351,1.547702,...,0.13,14651.249,1.754789,0.239545,-0.818309,-0.231220,-0.111108,-0.920169,-1.450523,-1.488785
2009-12-31,-1.406098,-1.421360,-1.415639,-1.435302,-1.435302,-2.236625,1.0,-1.604209,1.672052,1.503972,...,0.05,14651.249,1.239517,-0.028968,-1.263925,-0.109079,0.400549,0.030069,-1.414042,-1.453645
2010-03-31,-1.316705,-1.330820,-1.317794,-1.327305,-1.327305,0.395996,1.0,-1.603793,1.737335,1.503972,...,0.09,14651.249,0.118399,-0.392636,-1.094875,0.909172,0.588783,0.550334,-1.322440,-1.430618
2010-04-30,-1.264362,-1.283205,-1.296872,-1.316375,-1.316375,2.106263,1.0,-1.600326,1.964518,1.503972,...,0.20,14980.193,0.577313,-0.357636,0.522597,-0.761430,-1.315571,-1.607924,-1.282010,-1.321091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30,2.313944,2.302693,2.283023,2.276790,2.276790,0.195369,1.0,1.820540,-1.583370,-0.201501,...,0.05,23292.362,-0.598151,0.087515,1.050710,0.708475,0.804812,0.131750,2.278443,2.101319
2021-06-30,2.383814,2.381494,2.413298,2.414856,2.414856,0.157269,1.0,2.113773,-1.731344,-0.245232,...,0.08,23292.362,-0.318374,-0.221466,0.882336,-0.060083,-0.253436,0.520338,2.396579,2.406917
2021-08-31,2.494608,2.495961,2.522433,2.518025,2.518025,0.184247,1.0,2.199010,-1.917617,-0.463882,...,0.06,23828.973,-0.419014,-0.137248,0.820125,0.221487,0.505724,0.880479,2.491948,2.477216


In [39]:
# create LSTM model class
class LSTM_Model(nn.Module):
    def __init__(self, input_layer, hidden_layer, output_layer, num_layers=4, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.hidden_layer = hidden_layer
        self.num_layers = num_layers
        
        # LSTM layers with dropout
        self.lstm = nn.LSTM(input_layer, hidden_layer, num_layers, batch_first=True, dropout=dropout)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.linear_layer = nn.Linear(hidden_layer, output_layer)

    def forward(self, input_tensor):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        
        # Pass through LSTM layers
        out, _ = self.lstm(input_tensor, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out)
        
        # Pass through linear layer
        out = self.linear_layer(out[:, -1, :])
        return out

# create sequences for input data and corresponding labels
def create_sequence(input_data, sequence_length):
    sequences = []
    for i in range(0, len(input_data) - sequence_length):
        sequence = input_data[i : i + sequence_length, :-1]
        label = input_data[i + sequence_length, -1]
        sequences.append((sequence, label))
    return sequences

# train the model with data provided
def trainer(model, train_data, loss_func, opt, epochs):
    model.train()
    for epoch in range(epochs):
        for sequence, labels in train_data:
            opt.zero_grad()
            sequence = sequence.clone().detach().float()
            labels = labels.clone().detach().float().view(-1, 1)

            y = model(sequence)
            loss = loss_func(y, labels)
            loss.backward()
            opt.step()

        if epoch % 25 == 1:
            print(f'Epoch {epoch} loss: {loss.item():.4f}')

def predictor(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for sequence, _ in test_data:
            sequence = sequence.clone().detach().float()
            y = model(sequence)
            batch_predictions = torch.sigmoid(y)

            # Convert predictions to tensor if they are scalar or float
            if isinstance(batch_predictions, float) or batch_predictions.ndimension() == 0:
                batch_predictions = torch.tensor([batch_predictions])

            batch_predictions = torch.round(batch_predictions)

            predictions.extend(batch_predictions.squeeze().tolist())
    
    return predictions

# create sequences
sequence_length = 10
sequences = create_sequence(ticker[['Volume', 'CPI', 'Mortgage_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 'RSI (14D)', 
                                    '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'Target']].values, sequence_length)

# cross-validation
tscv = TimeSeriesSplit(n_splits=10)
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []

for fold, (train_index, test_index) in enumerate(tscv.split(sequences)):
    print(f'Fold {fold+1}')
    
    train_sequences = [sequences[i] for i in train_index]
    test_sequences = [sequences[i] for i in test_index]
    
    train_data = torch.utils.data.DataLoader(train_sequences, shuffle=True, batch_size=32)
    test_data = torch.utils.data.DataLoader(test_sequences, shuffle=False, batch_size=32)
    
    # initialize model
    model = LSTM_Model(input_layer=12, hidden_layer=10, output_layer=1)
    loss_func = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # train model
    epochs = 150
    trainer(model, train_data, loss_func, opt, epochs)
    
    # make predictions
    test_labels = [label for _, label in test_sequences]
    predictions = predictor(model, test_data)
    
    # calculate statistics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    kappa = cohen_kappa_score(test_labels, predictions)
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)
    
    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'Kappa: {kappa:.4f}\n')

# average scores across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_kappa = np.mean(kappas)

print(f'\nCross-Validation Results:')
print(f'Average Accuracy: {avg_accuracy:.4f}')
print(f'Average Precision: {avg_precision:.4f}')
print(f'Average Recall: {avg_recall:.4f}')
print(f'Average F1: {avg_f1:.4f}')
print(f'Average Kappa: {avg_kappa:.4f}')


Fold 1
Epoch 1 loss: 0.7096
Epoch 26 loss: 0.6896
Epoch 51 loss: 0.7008
Epoch 76 loss: 0.6812
Epoch 101 loss: 0.6704
Epoch 126 loss: 0.5633

Accuracy: 0.5000
Precision: 0.5000
Recall: 1.0000
F1: 0.6667
Kappa: 0.0000

Fold 2
Epoch 1 loss: 0.6996
Epoch 26 loss: 0.6999
Epoch 51 loss: 0.7049
Epoch 76 loss: 0.6751
Epoch 101 loss: 0.6853
Epoch 126 loss: 0.6766

Accuracy: 0.5000
Precision: 0.5000
Recall: 1.0000
F1: 0.6667
Kappa: 0.0000

Fold 3
Epoch 1 loss: 0.6934
Epoch 26 loss: 0.6882
Epoch 51 loss: 0.6669
Epoch 76 loss: 0.6977
Epoch 101 loss: 0.6855
Epoch 126 loss: 0.6857

Accuracy: 0.6250
Precision: 0.0000
Recall: 0.0000
F1: 0.0000
Kappa: 0.0000

Fold 4
Epoch 1 loss: 0.7021
Epoch 26 loss: 0.8657
Epoch 51 loss: 0.7508
Epoch 76 loss: 0.7549
Epoch 101 loss: 0.6863
Epoch 126 loss: 0.7036

Accuracy: 0.6250
Precision: 0.7500
Recall: 0.6000
F1: 0.6667
Kappa: 0.2500

Fold 5
Epoch 1 loss: 0.7354
Epoch 26 loss: 0.7245
Epoch 51 loss: 0.6754
Epoch 76 loss: 0.6928
Epoch 101 loss: 0.6318
Epoch 126 loss:

In [8]:
fall = (ticker.Target == 0).sum()
rise = (ticker.Target == 1).sum()

print(f'Number of days rising: {rise}')
print(f'Number of days falling: {fall}')

print(f'Rise % is: {(rise / (rise + fall)) * 100:.2f}%')
print(f'Fall % is: {(fall / (rise + fall)) * 100:.2f}%')

Number of days rising: 334
Number of days falling: 308
Rise % is: 52.02%
Fall % is: 47.98%


In [14]:
results = pd.DataFrame(columns=['Dataset', 'Average Accuracy', 'Average Precision', 
                                'Average Recall', 'Average F1', 'Average Kappa'])

datatset_name = (file_path.split('/')[-1].split('.')[0] + '_Monthly')

metrics = {'Dataset': datatset_name ,'Average Accuracy': avg_accuracy, 
           'Average Precision': avg_precision, 'Average Recall': avg_recall, 
           'Average F1': avg_f1, 'Average Kappa': avg_kappa}

export_results = results._append(metrics, ignore_index=True)

export_results.set_index('Dataset', inplace=True)

#export_results.to_csv(r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\IRP DATA\\" + datatset_name + '_monthly_w_macro.csv')

export_results

,Average Accuracy,Average Precision,Average Recall,Average F1,Average Kappa
Dataset,,,,,
^NYA_Monthly,0.482456,0.542963,0.463888,0.40118,-0.032521
